In [1]:
import sqlalchemy as db
from sqlalchemy import create_engine, inspect, MetaData

In [2]:
engine = db.create_engine('sqlite:///C:\\Users\\peter\\Documents\\Projekte\\Zähler\\tc_all_new.db', echo = False)

In [3]:
conn = engine.connect()

#### So stelle ist fest, welcher Driver verwendet wird...


In [4]:
engine.driver

'pysqlite'

#### ...und so komme ich an die Tabellen in der DB!

In [19]:
inspector = inspect(engine)

In [20]:
inspector.get_table_names()

['tcdata_CH_0020_01',
 'tcdata_CH_0020_02',
 'tcdata_CH_0020_03',
 'tcdata_CH_0020_04',
 'tcdata_CH_0020_05',
 'tcdata_CH_0020_06',
 'tcdata_CH_0020_07',
 'tcdata_CH_0020_08',
 'tcdata_CH_0084_01',
 'tcdata_CH_0084_02',
 'tcdata_CH_0084_03',
 'tcdata_CH_0084_04',
 'tcdata_CH_0150_01',
 'tcdata_CH_0150_02',
 'tcdata_CH_0240_01',
 'tcdata_CH_0240_02',
 'tcdata_CH_0240_03',
 'tcdata_CH_0240_04',
 'tcdata_CH_0240_05',
 'tcdata_CH_0240_06',
 'tcdata_CH_0240_07',
 'tcdata_CH_0240_08',
 'tcdata_CH_0243_01',
 'tcdata_CH_0243_02',
 'tcdata_CH_0243_03',
 'tcdata_CH_0243_04',
 'tcdata_CH_0274_01',
 'tcdata_CH_0274_02',
 'tcdata_CH_0274_03',
 'tcdata_CH_0274_04',
 'tcdata_CH_0274_05',
 'tcdata_CH_0274_06',
 'tcdata_CH_0282_01',
 'tcdata_CH_0282_02',
 'tcdata_CH_0282_03',
 'tcdata_CH_0282_04',
 'tcdata_CH_0282_05',
 'tcdata_CH_0282_06',
 'tcdata_CH_0286_01',
 'tcdata_CH_0286_02',
 'tcdata_CH_0286_03',
 'tcdata_CH_0286_04',
 'tcdata_CH_0286_05',
 'tcdata_CH_0286_06',
 'tcdata_CH_0286_07',
 'tcdata_C

In [21]:
tab = inspector.get_table_names()
tab = tab[:-1]

#### Und so die Spaltennamen und Typen in einer Tabelle!

In [22]:
columns_table = inspector.get_columns('tcdata_CH_0084_01') 


for c in columns_table :
    print(c['name'], c['type'])

time TEXT
vfr11 REAL
s12 REAL
vfr21 REAL
s22 REAL
vfr1 REAL
s2 REAL


#### Es ist garnicht so einfach, in SQLalchemy auf Tabellen einer bestehenden DB zuzugreifen. Bei Verwendung von des ORM kann "automate" zur Anwendung kommen. Ohne ORM geht es wie folgt:

In [23]:
metadata = db.MetaData(bind=engine)

In [24]:
table = db.Table('tcdata_CH_0084_01', metadata, autoload = True)

#### Voila, mit der so definierten Tabelle kann ich die Abfragen starten!

In [25]:
import pandas as pd

In [26]:
pd.options.display.max_rows=2000

In [27]:
pd.options.display.max_columns

20

### Abfrage 1 - Basics + Anzahl Datenpunkte

In [28]:
q = db.select(table)

In [29]:
result = conn.execute(q)

In [30]:
row = result.fetchmany(8)

In [31]:
row

[('2021-07-27 19:53:00+02:00', 900.0, 68.9000015, 0.0, 0.0, 0.0, 0.0),
 ('2021-07-27 19:58:00+02:00', 840.0, 65.5, 60.0, 73.0, 0.0, 0.0),
 ('2021-07-27 20:03:00+02:00', 840.0, 67.8000031, 60.0, 72.0, 0.0, 0.0),
 ('2021-07-27 20:08:00+02:00', 600.0, 79.3000031, 0.0, 0.0, 0.0, 0.0),
 ('2021-07-27 20:13:00+02:00', 720.0, 72.3000031, 0.0, 0.0, 0.0, 0.0),
 ('2021-07-27 20:18:00+02:00', 780.0, 73.9000015, 0.0, 0.0, 0.0, 0.0),
 ('2021-07-27 20:23:00+02:00', 780.0, 72.1999969, 0.0, 0.0, 0.0, 0.0),
 ('2021-07-27 20:28:00+02:00', 900.0, 69.5999985, 0.0, 0.0, 0.0, 0.0)]

In [33]:
from sqlalchemy.sql import func

In [34]:
q = db.select(func.count(table.c.time)).where(table.c.vfr11 == None)

In [35]:
result = conn.execute(q)
row = result.fetchmany(8)
row

[(196,)]

#### So greift man auf einzelne Spalten zu!

In [36]:
q = db.select(table.c.time, table.c.vfr11)

In [37]:
result = conn.execute(q)

In [38]:
row = result.fetchmany(8)

In [39]:
row

[('2021-07-27 19:53:00+02:00', 900.0),
 ('2021-07-27 19:58:00+02:00', 840.0),
 ('2021-07-27 20:03:00+02:00', 840.0),
 ('2021-07-27 20:08:00+02:00', 600.0),
 ('2021-07-27 20:13:00+02:00', 720.0),
 ('2021-07-27 20:18:00+02:00', 780.0),
 ('2021-07-27 20:23:00+02:00', 780.0),
 ('2021-07-27 20:28:00+02:00', 900.0)]

#### Funktionen

In [40]:
from sqlalchemy.sql import func

In [41]:
q = db.select(func.count(table.c.time))

In [42]:
result = conn.execute(q)
row = result.fetchmany(8)
row

[(170350,)]

In [43]:
q = db.select(func.max(table.c.s12))

In [44]:
result = conn.execute(q)
row = result.fetchmany(8)
row

[(144.0,)]

In [45]:
q = db.select(func.now())

In [46]:
result = conn.execute(q)
row = result.fetchmany(8)
row

[(datetime.datetime(2022, 7, 19, 8, 58, 10),)]

#### Try a loop

In [47]:
CountNull = []

In [48]:
for t in tab:
    table = db.Table(t, metadata, autoload = True)
    q = db.select(func.count(table.c.time))
    result = conn.execute(q)
    row = result.fetchmany(8)
    CountNull.append(row) 
    q = db.select(func.count(table.c.time)).where(table.c.vfr11 == None)
    result = conn.execute(q)
    row = result.fetchmany(8)
    CountNull.append(row)     
    q = db.select(func.count(table.c.time)).where(table.c.s12 > 120)
    result = conn.execute(q)
    row = result.fetchmany(8)
    CountNull.append(row)
    q = db.select(func.count(table.c.time)).where(table.c.s22 > 120)
    result = conn.execute(q)
    row = result.fetchmany(8)
    CountNull.append(row) 
    q = db.select(func.count(table.c.time)).where(table.c.s2 > 120)
    result = conn.execute(q)
    row = result.fetchmany(8)
    CountNull.append(row) 

In [49]:
print(CountNull)

[[(171367,)], [(20,)], [(4,)], [(0,)], [(0,)], [(171367,)], [(20,)], [(1509,)], [(9,)], [(0,)], [(171367,)], [(20,)], [(5549,)], [(55,)], [(0,)], [(171367,)], [(20,)], [(31600,)], [(24,)], [(0,)], [(171367,)], [(20,)], [(3881,)], [(0,)], [(0,)], [(171367,)], [(20,)], [(1340,)], [(0,)], [(0,)], [(171367,)], [(20,)], [(501,)], [(0,)], [(0,)], [(171367,)], [(20,)], [(13,)], [(0,)], [(0,)], [(170350,)], [(196,)], [(9,)], [(0,)], [(0,)], [(170350,)], [(196,)], [(62,)], [(0,)], [(0,)], [(170350,)], [(196,)], [(44,)], [(4,)], [(0,)], [(170350,)], [(196,)], [(11,)], [(5,)], [(0,)], [(170350,)], [(1304,)], [(5,)], [(10,)], [(0,)], [(170350,)], [(1315,)], [(3,)], [(3,)], [(0,)], [(170350,)], [(20,)], [(84,)], [(6,)], [(0,)], [(170350,)], [(20,)], [(330,)], [(0,)], [(0,)], [(170350,)], [(20,)], [(561,)], [(0,)], [(0,)], [(170350,)], [(20,)], [(1718,)], [(0,)], [(0,)], [(170350,)], [(20,)], [(662,)], [(0,)], [(0,)], [(170350,)], [(20,)], [(196,)], [(0,)], [(0,)], [(170350,)], [(20,)], [(144,)], [(

#### Das folgende wäre eine Basis dafür, Reihen zu löschen

In [ ]:
conn.execute(users.delete().where(users.c.name > 'm'))

#### Konjunktionen + Sortierung

In [51]:
from sqlalchemy import and_ , asc

In [52]:
q = db.select(table).where(and_(table.c.vfr11 == 1440, table.c.s12 > 75)).order_by(asc(table.c.s12))

In [53]:
result = conn.execute(q)
row = result.fetchmany(20)
row

[]

#### Gruppierung

In [54]:
q = db.select(table).where(and_(table.c.s12 >= 85)).group_by(table.c.vfr11)

In [55]:
result = conn.execute(q)
row = result.fetchmany(30)
row

[('2021-07-28 02:28:00+02:00', 60.0, 88.0, 0.0, 0.0, 0.0, 0.0),
 ('2021-08-04 07:16:00+02:00', 120.0, 92.0, 0.0, 0.0, 0.0, 0.0),
 ('2021-09-02 18:48:00+02:00', 180.0, 85.0, 0.0, 0.0, 0.0, 0.0)]

In [56]:
df = pd.read_sql_query(q, conn)

In [57]:
df.head(10)

,time,vfr11,s12,vfr21,s22,vfr1,s2
0,2021-07-28 02:28:00+02:00,60.0,88.0,0.0,0.0,0.0,0.0
1,2021-08-04 07:16:00+02:00,120.0,92.0,0.0,0.0,0.0,0.0
2,2021-09-02 18:48:00+02:00,180.0,85.0,0.0,0.0,0.0,0.0


#### Ab hier folgen Abfragen in normalem SQL

In [58]:
query ='''
SELECT  *, strftime('%Y', time, 'localtime') as Year, strftime('%m', time, 'localtime') as Month, strftime('%d', time, 'localtime') as Day, COUNT(time) as Anzahl
FROM tcdata_CH_0084_01
GROUP BY Year, Month
 '''

In [ ]:
df = pd.read_sql_query(query, conn)

In [ ]:
df.head(10)

### Abfrage 2 - Anzahl Datenpunkte mit Verkehrfluss vfr11

In [ ]:
query ='''
SELECT  strftime('%Y', time, 'localtime') as Year, strftime('%m', time, 'localtime') as Month, strftime('%d', time, 'localtime') as Day, COUNT(time) as Anzahl
FROM tcdata_CH_0084_01
WHERE vfr11 > 0
GROUP BY Year, Month
 '''

In [ ]:
df = pd.read_sql_query(query, conn)

In [ ]:
df.head(10)

In [ ]:
query ='''
SELECT  strftime('%Y', time, 'localtime') as Year, strftime('%m', time, 'localtime') as Month, strftime('%d', time, 'localtime') as Day, COUNT(time) as Anzahl
FROM tcdata_CH_0084_01
WHERE vfr11 IS NULL
GROUP BY Year, Month
 '''

In [ ]:
df = pd.read_sql_query(query, conn)

In [ ]:
df.head(10)

In [ ]:
query ='''
SELECT  strftime('%Y', time, 'localtime') as Year, strftime('%m', time, 'localtime') as Month, strftime('%d', time, 'localtime') as Day, COUNT(time) as Anzahl
FROM tcdata_CH_0084_01
WHERE vfr11 = 0
GROUP BY Year, Month
 '''

In [ ]:
df = pd.read_sql_query(query, conn)

In [ ]:
df.head(10)

### Abfrage 3 - Anzahl Datenpunkte mit Verkehrfluss vfr21

In [ ]:
query ='''
SELECT  strftime('%Y', time, 'localtime') as Year, strftime('%m', time, 'localtime') as Month, strftime('%d', time, 'localtime') as Day, COUNT(time) as Anzahl
FROM tcdata_CH_0084_01
WHERE vfr21 > 0
GROUP BY Year, Month
 '''

In [ ]:
df = pd.read_sql_query(query, conn)

In [ ]:
df.head(10)

In [ ]:
query ='''
SELECT  strftime('%Y', time, 'localtime') as Year, strftime('%m', time, 'localtime') as Month, strftime('%d', time, 'localtime') as Day, COUNT(time) as Anzahl
FROM tcdata_CH_0084_01
WHERE vfr21 IS NULL
GROUP BY Year, Month
 '''

In [ ]:
df = pd.read_sql_query(query, conn)

In [ ]:
df.head(10)

In [ ]:
query ='''
SELECT  strftime('%Y', time, 'localtime') as Year, strftime('%m', time, 'localtime') as Month, strftime('%d', time, 'localtime') as Day, COUNT(time) as Anzahl
FROM tcdata_CH_0084_01
WHERE vfr21 = 0
GROUP BY Year, Month
 '''

In [ ]:
df = pd.read_sql_query(query, conn)

In [ ]:
df.head(10)

### Abfrage 4 - Anzahl Datenpunkte mit Verkehrfluss vfr1

In [ ]:
query ='''
SELECT  strftime('%Y', time, 'localtime') as Year, strftime('%m', time, 'localtime') as Month, strftime('%d', time, 'localtime') as Day, COUNT(time) as Anzahl
FROM tcdata_CH_0084_01
WHERE vfr1 > 0
GROUP BY Year, Month
 '''

In [ ]:
df = pd.read_sql_query(query, conn)

In [ ]:
df.head(10)

In [ ]:
query ='''
SELECT  strftime('%Y', time, 'localtime') as Year, strftime('%m', time, 'localtime') as Month, strftime('%d', time, 'localtime') as Day, COUNT(time) as Anzahl
FROM tcdata_CH_0084_01
WHERE vfr1 IS NULL
GROUP BY Year, Month
 '''

In [ ]:
df = pd.read_sql_query(query, conn)

In [ ]:
df.head(10)

In [ ]:
query ='''
SELECT  strftime('%Y', time, 'localtime') as Year, strftime('%m', time, 'localtime') as Month, strftime('%d', time, 'localtime') as Day, COUNT(time) as Anzahl
FROM tcdata_CH_0084_01
WHERE vfr1 = 0
GROUP BY Year, Month
 '''

In [ ]:
df = pd.read_sql_query(query, conn)

In [ ]:
df.head(10)

### Dh es gibt für diesen Fahrzeugtyp keine Daten von "anderen Fahrzeugen"

### Nun zu statistischen Werten (zunächst mit 0 und NULL)

In [ ]:
query ='''
SELECT  strftime('%Y', time, 'localtime') as Year, strftime('%m', time, 'localtime') as Month, strftime('%d', time, 'localtime') as Day, COUNT(time) as Anzahl,
       ROUND(AVG(vfr11), 1) AS MEAN_Flow,
       MAX(vfr11) AS MAX_FLOW,
       MIN(vfr11) AS MIN_FLOW,
       ROUND(AVG(s12), 1) AS MEAN_Speed,
       MAX(s12) AS MAX_Speed,
       MIN(s12) AS MIN_Speed   
FROM tcdata_CH_0084_01
GROUP BY Year, Month
 '''

In [ ]:
df = pd.read_sql_query(query, conn)

In [ ]:
df.head(10)

### ...dann bereinigt!

In [ ]:
query ='''
SELECT  strftime('%Y', time, 'localtime') as Year, strftime('%m', time, 'localtime') as Month, strftime('%d', time, 'localtime') as Day, COUNT(time) as Anzahl,
       ROUND(AVG(vfr11), 1) AS MEAN_Flow,
       MAX(vfr11) AS MAX_FLOW,
       MIN(vfr11) AS MIN_FLOW,
       ROUND(AVG(s12), 1) AS MEAN_Speed,
       MAX(s12) AS MAX_Speed,
       MIN(s12) AS MIN_Speed   
FROM tcdata_CH_0084_01
WHERE vfr11 > 0
GROUP BY Year, Month
 '''

In [ ]:
df = pd.read_sql_query(query, conn)

In [ ]:
df.head(10)

### Weitere Detaillierung nach Tag

In [ ]:
query ='''
SELECT  strftime('%Y', time, 'localtime') as Year, strftime('%m', time, 'localtime') as Month, strftime('%d', time, 'localtime') as Day, COUNT(time) as Anzahl,
       ROUND(AVG(vfr11), 1) AS MEAN_Flow,
       MAX(vfr11) AS MAX_FLOW,
       MIN(vfr11) AS MIN_FLOW,
       ROUND(AVG(s12), 1) AS MEAN_Speed,
       MAX(s12) AS MAX_Speed,
       MIN(s12) AS MIN_Speed   
FROM tcdata_CH_0084_01
WHERE vfr11 > 0
GROUP BY Year, Month, Day
 '''

In [ ]:
df = pd.read_sql_query(query, conn)

In [ ]:
df.head(1000)

### Wechsel auf Analyse nach Wochentag

In [ ]:
query ='''
SELECT  strftime('%w', time, 'localtime') as Weekday, strftime('%Y', time, 'localtime') as Year, strftime('%m', time, 'localtime') as Month, strftime('%d', time, 'localtime') as Day, COUNT(time) as Anzahl,
       ROUND(AVG(vfr11), 1) AS MEAN_Flow,
       MAX(vfr11) AS MAX_FLOW,
       MIN(vfr11) AS MIN_FLOW,
       ROUND(AVG(s12), 1) AS MEAN_Speed,
       MAX(s12) AS MAX_Speed,
       MIN(s12) AS MIN_Speed   
FROM tcdata_CH_0084_01
WHERE vfr11 > 0
GROUP BY strftime('%w', time, 'localtime') 
 '''

In [ ]:
df = pd.read_sql_query(query, conn)

In [ ]:
df.head(10)

In [ ]:
query ='''
SELECT  strftime('%H', time, 'localtime') as Hour, strftime('%Y', time, 'localtime') as Year, strftime('%m', time, 'localtime') as Month, strftime('%d', time, 'localtime') as Day, COUNT(time) as Anzahl,
       ROUND(AVG(vfr11), 1) AS MEAN_Flow,
       MAX(vfr11) AS MAX_FLOW,
       MIN(vfr11) AS MIN_FLOW,
       ROUND(AVG(s12), 1) AS MEAN_Speed,
       MAX(s12) AS MAX_Speed,
       MIN(s12) AS MIN_Speed   
FROM tcdata_CH_0084_01
WHERE vfr11 > 0
GROUP BY strftime('%H', time, 'localtime') 
 '''

In [ ]:
df = pd.read_sql_query(query, conn)

In [ ]:
df.head(24)

In [ ]:
df.to_excel('84_01_hours.xls')